In [6]:
import numpy as np
import os
import soundfile as sf
import sys

sys.path.append("../src")
import localmodule

In [38]:
data_dir = localmodule.get_data_dir()
dataset_name = localmodule.get_dataset_name()
predictions_name = "_".join([dataset_name, "baseline-predictions"])
predictions_dir = os.path.join(data_dir, predictions_name)
recordings_name = "_".join([dataset_name, "full-audio"])
recordings_dir = os.path.join(data_dir, recordings_name)
sample_rate = localmodule.get_sample_rate()
clip_length = int(0.500 * sample_rate)
n_positives = 50

unit_str = "unit01"
prediction_path = os.path.join(predictions_dir, unit_str + ".npy")
prediction_matrix = np.load(prediction_path)
timestamps = prediction_matrix[:, 0]
odf = prediction_matrix[:, 1]
peak_locations = localmodule.pick_peaks(odf)
peak_values = odf[peak_locations]
peak_times = timestamps[peak_locations]
peak_samples = np.round(peak_times * sample_rate).astype("int")
sorting_indices = np.argsort(peak_values)[::-1]
sorted_peak_values = peak_values[sorting_indices]
sorted_peak_samples = peak_samples[sorting_indices]
recording_name = unit_str + ".flac"
recording_path = os.path.join(recordings_dir, recording_name)
full_night = sf.SoundFile(recording_path)
positives = []
for peak_sample in sorted_peak_samples[:n_positives]:
    begin_sample = peak_sample - int(clip_length / 2)
    full_night.seek(begin_sample)
    positive = full_night.read(clip_length)
    positives.append(positive)
    
import IPython.display
concatenated_positives = np.concatenate(positives)
IPython.display.Audio(data=concatenated_positives, rate=sample_rate)